# ETL-пайплайн
В прототипе ETL-скрипты запускаются вручную: через Jupyter Notebook или Python-скрипт
напрямую из консоли.
Подразумевается, что в боевой системе ETL-скрипты будут запускаться автоматически по
расписанию системой типа Airflow / Prefect или любой другой schedule-системой.

In [ ]:
from tools import *

logging.getLogger().setLevel('INFO')

## Загрузка сырых данных
Функции парсят входные файлы и директории - координаты станций, данные станций и профилемера - структурируют их и загружают в схему raw базы данных.
Пути к файлам указаны в константах в config.py

In [ ]:
upload_coordinates(PATH_STATION_COORDINATES)  # координаты станций и их названия
upload_station_metrics(PATH_STATION_METRICS)  # замеры станций
upload_profile(PATH_PROFILE)  # данные профилемера

## Обработка входных данных
Функции сопоставляют названиям файлов названия станций, проставляют уникальные идентификаторы.

In [ ]:
update_prod_station()  # обновляется справочник станций в production-схеме
update_filtered_metrics()  # замерам станций проставляются идетификаторы станций
update_filtered_profile()  # данным профилемера проставляется идентификатор станции

## Определение аномалий
Согласно конфигурации ALERT_CONFIG в config.py функция находит в указанном окне дат аномалии:
* Некорректные значения (нулевые/отрицательные)
* Значения в верхних/нижних перцентилях
ALERT_CONFIG - конфигурация, указывающая, что считать аномалиями для каких показателей

In [ ]:
define_anomaly(datetime(2020, 11, 1), datetime(2020, 11, 15))

## Получить предсказания
Для каждой метрики для каждой станции возвращает предсказываемые значения на сутки вперёд

In [ ]:
for metric_name in ['CO', 'NO', 'NO2', 'PM2.5', 'PM10']:
    get_predictions(metric_name)

## Обновить продакшн
Формирует из предсказаний и реальных замеров витрину для бекенда в схеме production

In [ ]:
update_prod()